## <b><font color='darkblue'>Smart home application</font></b>
考慮你在一家軟體公司上班, 這家公司最熱賣的產品的是 <b>SmartHome</b> App. 讓客戶可以有賓至如歸的感覺.

考慮下面目前產品的實作:

In [1]:
from typing import Protocol


# 電器實作
class Machine:
  def turn_on(self):
    print(f'Turn on {self.__class__.__name__}')
    
  def turn_off(self):
    print(f'Turn off {self.__class__.__name__}')
    

class Lamp(Machine):
  """ 檯燈 """
  
  def adjust_light(self, brightness: int):
    print(f'Adjust brightness to be {brightness}')
    

class AirCondr(Machine):
  """ 空調 """
    
  def adjust_temp(self, temp: int):
    print(f'Adjust temperature to {temp} degree')
    
    
class TV(Machine):
  """ 電視 """
    
  def switch_channel(self, channel: int):
    print(f'Switch channel to {channel}')

In [2]:
# 實例化 電器
lamp = Lamp()
air_condr = AirCondr()
tv = TV()

In [3]:
# 公司的 SmartHome app 的抽象類別
class SmartHome(Protocol):
  
  def __init__(self, lamp: Lamp, air_condr: AirCondr, tv: TV):
    self._lamp = lamp
    self._air_condr = air_condr
    self._tv = tv
    
  def come_home(self):
    pass
  
  def exit_home(self):
    pass

### <b><font color='darkgreen'>Customer A</font></b>
* **Come Home**
  - 打開檯燈
  - 打開冷氣 並 調到 25 度
* **Exit Home**
  - 關閉檯燈
  - 關閉冷氣

In [4]:
# 客戶 A
class SmartHomeA(SmartHome):
  
  def come_home(self):
    self._lamp.turn_on()
    self._air_condr.turn_on()
    self._air_condr.adjust_temp(25)
    
  def exit_home(self):
    self._lamp.turn_off()
    self._air_condr.turn_off()

接著來測試一下我們的產品 A:

In [5]:
smart_home = SmartHomeA(lamp, air_condr, tv)

In [6]:
# 回家
smart_home.come_home()

Turn on Lamp
Turn on AirCondr
Adjust temperature to 25 degree


In [7]:
# 離開家
smart_home.exit_home()

Turn off Lamp
Turn off AirCondr


### <b><font color='darkgreen'>Customer B</font></b>
* **Come Home**
  - 打開檯燈
  - 打開電視 並 轉到 65 台
* **Exit Home**
  - 關閉檯燈
  - 關閉電視

In [8]:
# 客戶 B
class SmartHomeB(SmartHome):
  
  def come_home(self):
    self._lamp.turn_on()
    self._tv.turn_on()
    self._tv.switch_channel(65)
    
  def exit_home(self):
    self._lamp.turn_off()
    self._tv.turn_off()

接著來測試一下我們的產品 B:

In [9]:
smart_home = SmartHomeB(lamp, air_condr, tv)

In [10]:
# 回家
smart_home.come_home()

Turn on Lamp
Turn on TV
Switch channel to 65


In [11]:
# 離開家
smart_home.exit_home()

Turn off Lamp
Turn off TV


### <b><font color='darkgreen'>Requirement - TV 升級</font></b>
客戶 B 要求升級 <b><font color='blue'>TV</font></b> 成 <b><font color='blue'>TV_Toshiba</font></b>

In [12]:
class TV_Toshiba(Machine):
  """ Toshiba 電視 """
    
  def change_channel(self, channel: int):
    print(f'Change channel to {channel}')

In [13]:
tv = TV_Toshiba()
smart_home = SmartHomeB(lamp, air_condr, tv)

In [14]:
# 回家 -> 爆炸 -> 客訴 ><"
smart_home.come_home()

Turn on Lamp
Turn on TV_Toshiba


AttributeError: 'TV_Toshiba' object has no attribute 'switch_channel'

### <b><font color='darkgreen'>Requirement - 支援新電器</font></b>
客戶 A 希望支援新電器音響 <b><font color='darkblue'>JBLBluetoothSpeaker</font></b>:

In [15]:
class JBLBluetoothSpeaker(Machine):
  """ 藍芽喇叭 """
  
  def adjust_volume(self, volume: int):
    print(f'Adjust volume to be {volume}')

In [16]:
# 再次爆炸 -> 客訴 T_T
speaker = JBLBluetoothSpeaker()
smart_home = SmartHomeA(lamp, air_condr, tv, speaker)

TypeError: SmartHome.__init__() takes 4 positional arguments but 5 were given

## <b><font color='darkblue'>DP - Command</font></b>
總算進入到今天的主題 - [**Design pattern Command**](https://en.wikipedia.org/wiki/Command_pattern):
> In object-oriented programming, the **command pattern** is a [behavioral design pattern](https://en.wikipedia.org/wiki/Behavioral_pattern) in which <b>an object is used to encapsulate all information needed to perform an action or trigger an event at a later time</b>. This information includes the method name, the object that owns the method and values for the method parameters.

這是什麼意思？當Ａ要請求Ｂ執行任務時，Ａ會呼叫Ｂ然後Ｂ在完成任務，在這種情況下Ａ需直接和Ｂ進行溝通，就像Ａ是老闆，他要交代員工Ｂ去做事情一樣，如下圖。我們稱做 Ａ 為 <b>Invoker/Sender</b>，Ｂ為 <b>Reciver</b>。底下為此 pattern 的 UML class diagram:
![class diagram](images/2.PNG)

* **Command**:	用來宣告執行操作的interface / abstract class。
* **ConcreteCommand**:	**Command** 的實體物件，通常會持有 Receiver，並呼叫 Receiver 的功能來完成命令要執行的操作。
* **Receiver**(接收者):	幹活的角色， 命令傳遞到被執行。
* **Invoker**(請求者):	接收並要求執行命令。
* **Client**(裝配者):	建立 Command Object，組裝 Command Object 和 Receiver

### <b><font color='darkgreen'>Smart Home App Refactoring</font></b>
在重構 <b>SmartHome</b> App 之前, 我們有一些準備工作:

In [34]:
class Command(Protocol):
  def execute(self):
    pass
  
  
class TurnOnTVCmd(Command):
  def __init__(self, tv:TV, channel: int):
    self._tv = tv
    self._channel = channel
    
  def execute(self):
    self._tv.turn_on()
    self._tv.switch_channel(self._channel)
    
  def undo(self):
    self._tv.turn_off()
    
    
class TurnOnToshibaTVCmd(Command):
  def __init__(self, tv:TV_Toshiba, channel: int):
    self._tv = tv
    self._channel = channel
    
  def execute(self):
    self._tv.turn_on()
    self._tv.change_channel(self._channel)
    
  def undo(self):
    self._tv.turn_off()
    
    
class TurnOnLamp(Command):
  def __init__(self, lamp: Lamp):
    self._lamp = lamp
    
  def execute(self):
    self._lamp.turn_on()
    
  def undo(self):
    self._lamp.turn_off()
    
    
class TurnOnAirCondr(Command):
  def __init__(self, air_condr: AirCondr, temp: int):
    self._air_condr = air_condr
    self._temp = temp
    
  def execute(self):
    self._air_condr.turn_on()
    self._air_condr.adjust_temp(self._temp)

  def undo(self):
    self._air_condr.turn_off()
    
    
class TurnOnJBLSpeaker(Command):
  def __init__(self, speaker: JBLBluetoothSpeaker, volume: int):
    self._speaker = speaker
    self._volume = volume
    
  def execute(self):
    self._speaker.turn_on()
    self._speaker.adjust_volume(self._volume)
    
  def undo(self):
    self._speaker.turn_off()

In [25]:
cmd = TurnOnTVCmd(TV(), 65)
cmd.execute()

Turn on TV
Switch channel to 65


In [26]:
cmd.undo()

Turn off TV


In [30]:
cmd = TurnOnToshibaTVCmd(TV_Toshiba(), 65)
cmd.execute()

Turn on TV_Toshiba
Change channel to 65


In [31]:
cmd.undo()

Turn off TV_Toshiba


接著來重構我們的 <b>SmartHome</b>

In [37]:
# 公司的 SmartHome app 類別
class SmartHome(Protocol):
  
  def __init__(self, commands: list[Command]):
    self._commands = commands
    
  def come_home(self):
    for cmd in self._commands:
      cmd.execute()
  
  def exit_home(self):
    for cmd in self._commands:
      cmd.undo()

### <b><font color='darkgreen'>Customer A</font></b>
* **Come Home**
  - 打開檯燈
  - 打開冷氣 並 調到 25 度
* **Exit Home**
  - 關閉檯燈
  - 關閉冷氣

In [38]:
smart_home = SmartHome([
  TurnOnLamp(lamp),
  TurnOnAirCondr(air_condr, 25),
])

In [39]:
# 回家
smart_home.come_home()

Turn on Lamp
Turn on AirCondr
Adjust temperature to 25 degree


In [40]:
# 離開家
smart_home.exit_home()

Turn off Lamp
Turn off AirCondr


### <b><font color='darkgreen'>Customer B</font></b>
* **Come Home**
  - 打開檯燈
  - 打開電視 並 轉到 65 台
* **Exit Home**
  - 關閉檯燈
  - 關閉電視

In [44]:
smart_home = SmartHome([
  TurnOnLamp(lamp),
  TurnOnTVCmd(TV(), 65),
])

In [45]:
# 回家
smart_home.come_home()

Turn on Lamp
Turn on TV
Switch channel to 65


In [46]:
# 離開家
smart_home.exit_home()

Turn off Lamp
Turn off TV


### <b><font color='darkgreen'>Customer B with Toshiba TV</font></b>
* **Come Home**
  - 打開檯燈
  - 打開 Toshiba 電視 並 轉到 65 台
* **Exit Home**
  - 關閉檯燈
  - 關閉電視

In [48]:
smart_home = SmartHome([
  TurnOnLamp(lamp),
  TurnOnToshibaTVCmd(TV_Toshiba(), 65),
])

In [49]:
# 回家
smart_home.come_home()

Turn on Lamp
Turn on TV_Toshiba
Change channel to 65


In [50]:
# 離開家
smart_home.exit_home()

Turn off Lamp
Turn off TV_Toshiba


### <b><font color='darkgreen'>Customer A with Speaker</font></b>
* **Come Home**
  - 打開檯燈
  - 打開冷氣 並 調到 25 度
  - 打開喇叭 並 調聲量到 30
* **Exit Home**
  - 關閉檯燈
  - 關閉冷氣
  - 關閉喇叭

In [52]:
smart_home = SmartHome([
  TurnOnLamp(lamp),
  TurnOnAirCondr(air_condr, 25),
  TurnOnJBLSpeaker(JBLBluetoothSpeaker(), 30),
])

In [53]:
# 回家
smart_home.come_home()

Turn on Lamp
Turn on AirCondr
Adjust temperature to 25 degree
Turn on JBLBluetoothSpeaker
Adjust volume to be 30


In [54]:
# 離開家
smart_home.exit_home()

Turn off Lamp
Turn off AirCondr
Turn off JBLBluetoothSpeaker


## <b><font color='darkblue'>Supplement</font></b>
* [iT邦幫忙 - 初探設計模式 - 命令模式 ( Command Pattern )](https://ithelp.ithome.com.tw/articles/10204425)
```
命令模式有幾個優點：
* 它能較容易的設計一個命令序列。
* 在需要的狀況下，可以較容易的將命令記入日誌。
* 允許接收請求的一方決定是否要否決請求。
* 可以容易的實現對請求的取消和重做。
* 由於加進新的具體命令類別不影響其他類別，因此增加新的具體命令類別很容易。

最後、最大的優點是將請求的物件和執行的物件分開。
```

* [Refactoring Guru - Command pattern](https://refactoring.guru/design-patterns/command)
> <font size='3ptx'><b>Command is a behavioral design pattern</b> that turns a request into a stand-alone object that contains all information about the request</font>. This transformation lets you pass requests as a method arguments, delay or queue a request’s execution, and support undoable operations.
![analog](images/1.PNG)

* [Design Pattern 系列文章導讀 - 命令模式 | Command Pattern](https://ianjustin39.github.io/ianlife/design-pattern/command-pattern/)
> 老闆(Sender)將任務封裝成備忘錄(Command)，然後秘書(Invoker)再經由備忘錄的工作事項分派任務給員工(Receiver)。如此一來老闆不需要知道是哪個員工執行，只需要秘書回報任務結果即可。

* [Medium - Design Pattern: Behavioral Patterns — Command Pattern (命令模式)](https://medium.com/bucketing/behavioral-patterns-command-pattern-7e531c929fc3)
> Chain of Responsibility將邏輯拆開，透過物件之間的連接做呼叫，物件間的聯就透過建構者或Dependency Inject注入。Commande Pattern比較偏向將一個物件原本的 method 拆解成一個個的 Command，而 Chain of Responsibility 的 Handler 粒度更大一點，更像是一個個完整邏輯的物件。